In [1]:
# Import library
import pickle
import cv2 as cv
import numpy as np
from sklearn.preprocessing import LabelEncoder
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet

In [2]:
# Load Saved Model
with open('fix_model_facenet_160x160.pkl', 'rb') as f:
    fix_model_facenet = pickle.load(f)

In [3]:
# Initiate MTCNN Model
img_detector = MTCNN()

In [4]:
# Load the image
t_img = cv.imread("data/abed/abed_2.jpg")

# Convert the image to RGB
t_img = cv.cvtColor(t_img, cv.COLOR_BGR2RGB)

# Extract the coordinates and size of the bounding box from the first detection result
x,y,w,h = img_detector.detect_faces(t_img)[0]['box']

1/1 [==============================] - 0s 117ms/step


In [5]:
# Instantiate the FaceNet model
embedder = FaceNet()

# Define a function to get the embedding of a face image
def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0) 
    # 4D (Nonex160x160x3)
    yhat= embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)

In [6]:
# Crop the detected face region from t_img using the bounding box coordinates
t_img = t_img[y:y+h, x:x+w]

# Resize the cropped face image to 160x160
t_img = cv.resize(t_img, (160,160))

# Get the embedding (feature vector) for the resized face image using the get_embedding function (Facenet)
test_img = get_embedding(t_img)

1/1 [==============================] - 2s 2s/step


In [7]:
# Define the labels for the prediction classes
LABEL = ['abed', 'gibran', 'iyal', 'vicky']

# Convert the test image to a numpy array
test_img = np.array(test_img)

# Flatten the test image array to a 1D array
test_img_flat = test_img.reshape(1, -1)

# Use the pre-trained FaceNet model to predict the class of the test image
prediction = fix_model_facenet.predict(test_img_flat)

# Find the index of the class with the highest predicted probability
pred_idx = np.argmax(prediction)
# Map the predicted index to the corresponding label
pred_label = LABEL[pred_idx]

# Print the result of the prediction
print(f"Prediction result: {pred_label}")

Prediction result: abed
